In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install accelerate  # 세션 다시 시작 필요

In [ ]:
pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [ ]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install peft

In [ ]:
!nvidia-smi

Tue Mar  5 02:40:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import random
import os
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
# data_path = './data/'
data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'

In [ ]:
# train= pd.read_csv(f'{data_path}train.csv')
test = pd.read_csv(f'{data_path}test.csv')
train = pd.read_csv(f'{data_path}train.csv')
# train_qna = pd.read_csv(f'{data_path}train_80000_rule_based.csv')

train_ft = pd.read_csv(f'{data_path}train_ft_81900.csv')

In [ ]:
category_lst = []
question_lst = []
answer_lst = []
for i in range(1, 3):
    for j in range(1, 6):
        for n in range(train.shape[0]):
            category = train['category'][n]
            question = train[f'질문_{i}'][n]
            answer = train[f'답변_{j}'][n]
            category_lst.append(category)
            question_lst.append(question)
            answer_lst.append(answer)

qna_df = pd.DataFrame({'category':category_lst,
                       'question':question_lst,
                       'answer':answer_lst})
qna_df

,category,question,answer
0,건축구조,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,건축구조,내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,건축구조,철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,건축구조,철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ..."
4,건축구조,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.
...,...,...,...
6435,타 마감하자,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
6436,타 마감하자,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
6437,타 마감하자,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
6438,타 마감하자,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


# Model Inference

## Original Similarity

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
# bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
# bert_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
bert_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')


def evaluate_similarity(input_text, generated_text, alpha=0.95): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

- jhgan/ko-sroberta-multitask

In [ ]:
# 임베딩 모델 jhgan/ko-sroberta-multitask

# paire wise 사용
evaluate_similarity("방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?",
                    "방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목마루의 단점 중 하나는 가격이 비싸다는 것입니다. 또한, 원목이 두께가 있어서 열전도율이 낮아 일반적으로 바닥 난방과 같은 열원을 사용할 경우 효과가 떨어질 수 있고, 습기에 약하여 습도 조절에 민감하다는 점도 단점으로 꼽힙니다.", 1)

0.736335813999176

In [ ]:
# 임베딩 모델 jhgan/ko-sroberta-multitask

cosine_score("방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?",
                    "방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목마루의 단점 중 하나는 가격이 비싸다는 것입니다. 또한, 원목이 두께가 있어서 열전도율이 낮아 일반적으로 바닥 난방과 같은 열원을 사용할 경우 효과가 떨어질 수 있고, 습기에 약하여 습도 조절에 민감하다는 점도 단점으로 꼽힙니다.")

0.7363358

- sentence-transformers/paraphrase-multilingual-mpnet-base-v2

In [ ]:
# 임베딩 모델 sentence-transformers/paraphrase-multilingual-mpnet-base-v2

# paire wise 사용
evaluate_similarity("방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?",
                    "방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목마루의 단점 중 하나는 가격이 비싸다는 것입니다. 또한, 원목이 두께가 있어서 열전도율이 낮아 일반적으로 바닥 난방과 같은 열원을 사용할 경우 효과가 떨어질 수 있고, 습기에 약하여 습도 조절에 민감하다는 점도 단점으로 꼽힙니다.")

0.6260981538620861

In [ ]:
# 임베딩 모델 sentence-transformers/paraphrase-multilingual-mpnet-base-v2

cosine_score("방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?",
                    "방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목마루의 단점 중 하나는 가격이 비싸다는 것입니다. 또한, 원목이 두께가 있어서 열전도율이 낮아 일반적으로 바닥 난방과 같은 열원을 사용할 경우 효과가 떨어질 수 있고, 습기에 약하여 습도 조절에 민감하다는 점도 단점으로 꼽힙니다.")

0.65809363

- distiluse-base-multilingual-cased-v1


In [ ]:
# 임베딩 모델 distiluse-base-multilingual-cased-v1 (데이콘 제공)

# paire wise 사용
evaluate_similarity("방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?",
                    "방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목마루의 단점 중 하나는 가격이 비싸다는 것입니다. 또한, 원목이 두께가 있어서 열전도율이 낮아 일반적으로 바닥 난방과 같은 열원을 사용할 경우 효과가 떨어질 수 있고, 습기에 약하여 습도 조절에 민감하다는 점도 단점으로 꼽힙니다.")


0.19076950040730561

In [ ]:
# 임베딩 모델 distiluse-base-multilingual-cased-v1 (데이콘 제공)

cosine_score("방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?",
                    "방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목마루의 단점 중 하나는 가격이 비싸다는 것입니다. 또한, 원목이 두께가 있어서 열전도율이 낮아 일반적으로 바닥 난방과 같은 열원을 사용할 경우 효과가 떨어질 수 있고, 습기에 약하여 습도 조절에 민감하다는 점도 단점으로 꼽힙니다.")

0.19985306

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score

# embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')



In [ ]:
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

def cosine_score(input_text, generated_text):
    input_embedding = embedding_model.encode(input_text)
    generated_embedding = embedding_model.encode(generated_text)

    cosine_similarity_score = cosine_similarity(input_embedding, generated_embedding)
    cosine_similarity_score = max(cosine_similarity_score, 0)

    return cosine_similarity_score

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_similarity(query, sentence):
    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([query, sentence])

    # 코사인 유사도 계산
    from sklearn.metrics.pairwise import cosine_similarity
    similarity_score = cosine_similarity(vectors[0], vectors[1])[0][0]

    return similarity_score


In [ ]:
def jaccard_similarity(vector1, vector2):
    set1 = set(vector1)
    set2 = set(vector2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    similarity = intersection / union
    return similarity

def jaccard_score(input_text, generated_text):
    input_embedding = embedding_model.encode(input_text).tolist()
    generated_embedding = embedding_model.encode(generated_text).tolist()
    jaccard_similarity_score = jaccard_similarity(input_embedding, generated_embedding)
    return jaccard_similarity_score

In [ ]:
import numpy as np

def euclidean_similarity(vector1, vector2):
    distance = np.linalg.norm(vector1 - vector2)
    similarity = 1 / (1 + distance)
    return similarity

def euclidean_score(input_text, generated_text):
    input_embedding = embedding_model.encode(input_text)
    generated_embedding = embedding_model.encode(generated_text)
    euclidean_similarity_score = euclidean_similarity(input_embedding, generated_embedding)
    return euclidean_similarity_score

## 모델별 추론 비교

In [ ]:
kogpt_tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
# kogpt_model_name = 'kogpt2_80000_5epoch_0.13981loss'
# kogpt_model_1 = AutoModelForCausalLM.from_pretrained(f'{data_path}{kogpt_model_name}').to(device)

In [ ]:
kogpt_model_name = 'kogpt2_89000_0304_5epoch_0.12447loss'
kogpt_model_2 = AutoModelForCausalLM.from_pretrained(f'{data_path}model/{kogpt_model_name}').to(device)

In [ ]:
kogpt_model_name = 'kogpt2_89000_0304_10epoch_0.10532loss'
kogpt_model_3 = AutoModelForCausalLM.from_pretrained(f'{data_path}model/{kogpt_model_name}').to(device)

In [ ]:
kogpt_model_name = 'kogpt2_trial7_date_2_26_epoch8_loss0.1487'
kogpt_model_4 = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{kogpt_model_name}').to(device)

In [ ]:
kogpt2_80000_5epoch_0.18720loss

kogpt2_213486_8epoch_0.13773loss

kogpt2_213486_10epoch_0.13645loss

kogpt2_213486_5epoch_0.14188loss

In [ ]:
kogpt_model_name = 'kogpt2_80000_5epoch_0.18720loss'
kogpt_80_5ep = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{kogpt_model_name}').to(device)

In [ ]:
def Chatbot(input_text, model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    text = input_text.strip()
    print('Question', text)

    text = '<q>' + text + '</s><a>'
    x = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    generated_texts = []  # 답변 후보군을 담을 리스트
    for i in range(num_samples):
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    # repetition_penalty=1.0, # Rule-based 방식 사용하려면 off 해야 함
                                    num_return_sequences=1,
                                    do_sample=True,
                                    temperature=0.87,
                                    top_k=27,
                                    top_p=0.7)
        # result_ids = model.generate(x,max_length=max_len,num_beams=1,num_return_sequences=1,no_repeat_ngram_size=4,use_cache=True,do_sample=True,temperature=0.9,top_k=90,top_p=0.95,early_stopping=True)

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = generated_text[q_len:-4]

        similarity_score = evaluate_similarity(text, generated_text)
        generated_texts.append((similarity_score,generated_text))
        print(generated_texts[i]) # 실제 돌릴땐 off

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    print('Answer')
    return best_generated_text

In [ ]:
def Similarity_Comparison_Chatbot(input_text, model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    text = input_text.strip()
    print('Question', text)

    text = '<q>' + text + '</s><a>'
    x = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    generated_texts = []  # 답변 후보군을 담을 리스트
    similarity_scores = []

    for i in range(num_samples):
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    # repetition_penalty=1.0, # Rule-based 방식 사용하려면 off 해야 함
                                    num_return_sequences=1,
                                    do_sample=True,
                                    temperature=0.87,
                                    top_k=27,
                                    top_p=0.7)
        # result_ids = model.generate(x,max_length=max_len,num_beams=1,num_return_sequences=1,no_repeat_ngram_size=4,use_cache=True,do_sample=True,temperature=0.9,top_k=90,top_p=0.95,early_stopping=True)

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = generated_text[q_len:-4]

        cosine_similarity_score = cosine_score(text, generated_text)
        tfidf_similarity_score = tfidf_similarity(text, generated_text)
        euclidean_similarity_score = euclidean_score(text, generated_text)
        similarity_score = evaluate_similarity(text, generated_text)

        generated_texts.append((similarity_score,cosine_similarity_score, generated_text))
        similarity_scores.append((cosine_similarity_score, tfidf_similarity_score, euclidean_similarity_score))

        print('Answer', generated_texts[i]) # 실제 돌릴땐 off
        # print('Cosine-TFIDF-Euclidean', similarity_scores[i])


        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    print('Answer')
    return best_generated_text

In [ ]:
kogpt_model_2

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

#### 임베딩 모델 jhgan/ko-sroberta-multitask

In [ ]:
reset_seeds(SEED)

max_len = 200 ## len 200일때가 더 나음 ?

print('kogpt_model_2')
print(Similarity_Comparison_Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_2, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_3')
print(Similarity_Comparison_Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_3, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_4')
print(Similarity_Comparison_Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_4, kogpt_tokenizer, max_len, device))
print(' ')

# 점수는 유사도(jacc0.05), 코사인 유사도

kogpt_model_2
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Answer (0.5969254076808364, 0.6420057, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이언트에게 건축 재료의 선택과 함께 추가적인 비용과 유지보수에 대해 충분한 설명을 제공해야 합니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는데, 이는 건강한 실내 환경을 유지하는 데 도움을 줍니다. 또한, 열전도율이 가장 낮아서 효율적인 단열 효과를 제공하며 시공이 간편하여 공사 기간을 단축하는 장점이 있습니다.')
Answer (0.5969254076808364, 0.6420057, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이언트에게 건축 재료의 선택과 함께 추가적인 비용과 유지보수에 대해 충분한 설명을 제공해야 합니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는데, 이는 건강한 실내 환경을 유지하는 데 도움을 줍니다. 또한, 열전도율이 가장 낮아서 효율적인 단열 효과를 제공하며 시공이 간편하여 공사 기간을 단축하는 장점이 있습니다.')
Answer (0.598244462940428, 0.6659769, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 

임베딩 모델 distiluse-base-multilingual-cased-v1

In [ ]:
reset_seeds(SEED)

max_len = 200 ## len 200일때가 더 나음 ?

print('kogpt_model_2')
print(Similarity_Comparison_Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_2, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_3')
print(Similarity_Comparison_Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_3, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_4')
print(Similarity_Comparison_Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_4, kogpt_tokenizer, max_len, device))
print(' ')

# 점수는 유사도(jacc0.05), 코사인 유사도

kogpt_model_2
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Answer (0.2754910976022154, 0.28941226, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이언트에게 건축 재료의 선택과 함께 추가적인 비용과 유지보수에 대해 충분한 설명을 제공해야 합니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는데, 이는 건강한 실내 환경을 유지하는 데 도움을 줍니다. 또한, 열전도율이 가장 낮아서 효율적인 단열 효과를 제공하며 시공이 간편하여 공사 기간을 단축하는 장점이 있습니다.')
Answer (0.2754910976022154, 0.28941226, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이언트에게 건축 재료의 선택과 함께 추가적인 비용과 유지보수에 대해 충분한 설명을 제공해야 합니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는데, 이는 건강한 실내 환경을 유지하는 데 도움을 줍니다. 또한, 열전도율이 가장 낮아서 효율적인 단열 효과를 제공하며 시공이 간편하여 공사 기간을 단축하는 장점이 있습니다.')
Answer (0.2754972026083204, 0.28941226, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용

In [ ]:
reset_seeds(SEED)

max_len = 200 ## len 200일때가 더 나음 ?

print('kogpt_model_2')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_2, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_3')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_3, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_4')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_4, kogpt_tokenizer, max_len, device))
print(' ')

kogpt_model_2
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
(0.5969254076808364, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이언트에게 건축 재료의 선택과 함께 추가적인 비용과 유지보수에 대해 충분한 설명을 제공해야 합니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는데, 이는 건강한 실내 환경을 유지하는 데 도움을 줍니다. 또한, 열전도율이 가장 낮아서 효율적인 단열 효과를 제공하며 시공이 간편하여 공사 기간을 단축하는 장점이 있습니다.')
(0.5969254076808364, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이언트에게 건축 재료의 선택과 함께 추가적인 비용과 유지보수에 대해 충분한 설명을 제공해야 합니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는데, 이는 건강한 실내 환경을 유지하는 데 도움을 줍니다. 또한, 열전도율이 가장 낮아서 효율적인 단열 효과를 제공하며 시공이 간편하여 공사 기간을 단축하는 장점이 있습니다.')
(0.598244462940428, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이

In [ ]:
reset_seeds(SEED)

max_len = 200 ## len 200일때가 더 나음 ?

print('kogpt_model_2')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_2, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_3')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_3, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_4') # 실험 채택!!
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_4, kogpt_tokenizer, max_len, device))
print(' ')

kogpt_model_2
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
(0.5969254076808364, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이언트에게 건축 재료의 선택과 함께 추가적인 비용과 유지보수에 대해 충분한 설명을 제공해야 합니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는데, 이는 건강한 실내 환경을 유지하는 데 도움을 줍니다. 또한, 열전도율이 가장 낮아서 효율적인 단열 효과를 제공하며 시공이 간편하여 공사 기간을 단축하는 장점이 있습니다.')
(0.5969254076808364, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이언트에게 건축 재료의 선택과 함께 추가적인 비용과 유지보수에 대해 충분한 설명을 제공해야 합니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는데, 이는 건강한 실내 환경을 유지하는 데 도움을 줍니다. 또한, 열전도율이 가장 낮아서 효율적인 단열 효과를 제공하며 시공이 간편하여 공사 기간을 단축하는 장점이 있습니다.')
(0.598244462940428, '큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이

# Model Evaluation

## 1) Perplexity
- https://wikidocs.net/21697

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np

def calculate_perplexity(model, tokenizer, test_data):
    total_log_probability = 0.0
    total_words = 0

    for text in test_data:
        input_ids = tokenizer.encode(text, return_tensors="pt")
        with torch.no_grad():
            outputs = model(input_ids).to(device)

        logits = outputs.logits
        word_probabilities = torch.nn.functional.softmax(logits, dim=-1)
        word_probabilities = word_probabilities.squeeze(0)  # 첫 번째 차원 제거

        for word_id in input_ids.squeeze(0):
            total_log_probability += torch.log(word_probabilities[word_id])
            total_words += 1

    average_log_probability = total_log_probability / total_words
    perplexity = np.exp(-average_log_probability)

    return perplexity

In [ ]:
calculate_perplexity(kogpt_model_4.to(device), kogpt_tokenizer, test['질문'][0])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
import torch
import numpy as np

def calculate_perplexity_for_samples(model, tokenizer, text_samples, device):
    total_log_probability = 0.0
    total_words = 0

    for text in text_samples:
        input_text = '<q>' + text + '</s><a>'
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
        q_len = len(input_text) + 1

        with torch.no_grad():
            outputs = model.generate(input_ids,
                                    max_length=100,  # 적절한 max_length 값으로 조정
                                    num_return_sequences=1,
                                    do_sample=True,
                                    temperature=0.87,
                                    top_k=27,
                                    top_p=0.7)

        generated_text = tokenizer.decode(outputs[0])[q_len:-4]

        for word_id in input_ids.squeeze(0):
            # 확률 값을 직접 추출하여 사용
            word_probabilities = torch.nn.functional.softmax(outputs[0], dim=-1)
            word_probability = word_probabilities[0, torch.tensor(word_id, dtype=torch.long)]  # dtype 변환

            total_log_probability += torch.log(word_probability)
            total_words += 1

    average_log_probability = total_log_probability / total_words
    perplexity = np.exp(-average_log_probability)

    return perplexity

# 사용 예시
input_text = "방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?"
num_samples = 5
text_samples = [input_text] * num_samples

perplexity = calculate_perplexity_for_samples(kogpt_model_4, kogpt_tokenizer, text_samples, device)
print(f"Perplexity for samples: {perplexity}")



RuntimeError: "host_softmax" not implemented for 'Long'

In [ ]:
import numpy as np

def calculate_perplexity(model, test_data):
    total_log_probability = 0.0
    total_words = 0

    for sequence in test_data:
        for word_index in sequence:
            # 모델에서 단어의 확률을 얻어옴 (예: model.predict 함수 사용)
            word_probability = model.predict(word_index)

            # 확률의 로그를 취하여 누적
            total_log_probability += np.log(word_probability)

            # 전체 단어 수 증가
            total_words += 1

    # 평균 로그 우도 계산
    average_log_probability = total_log_probability / total_words

    # perplexity 계산
    perplexity = np.exp(-average_log_probability)

    return perplexity


## 2) Qualitative - SSA

# Solar Model + KoGPT

## 1) Solar Zeroshot Model

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# from peft import LoraConfig, PeftModel
# from trl import SFTTrainer
import json
import torch

# 모델 식별자
model_ID = "megastudyedu/M-SOLAR-10.7B-v1.3"

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    use_nested_quant=False
)

# AdapterModelForCausalLM 클래스로 모델 정의
base_model = AutoModelForCausalLM.from_pretrained(
    model_ID,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [ ]:
solar_tokenizer = AutoTokenizer.from_pretrained(model_ID)

In [ ]:
# 학습 전 peft 모델
print(base_model)
print('#'*100)

# 학습 후 peft 모델
adapter_model_name = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/models/megast_0302/'
trained_solar_model = PeftModel.from_pretrained(base_model, adapter_model_name).to(device)
print(trained_solar_model)
print('#'*100)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
   

In [ ]:
def generate_response(user_request, model, tokenizer, max_len=1000):
    conversation = [ {'role': 'user', 'content': user_request} ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             use_cache=True,
                             max_length=max_len,
                            num_return_sequences=1,
                            do_sample=True,
                            temperature=0.87,
                            top_k=27,
                            top_p=0.7
                             )
    output_text = tokenizer.decode(outputs[0])
    output_text = output_text.split("### Assistant:")[1].strip()
    output_text = output_text.replace('\n','').replace('</s>','')

    return output_text

In [ ]:
generate_response(test['질문'][0],base_model, solar_tokenizer, 400)

'방청 페인트는 방청 성분이 포함된 특별한 종류의 페인트입니다. 방청 성분은 철과 같은 금속 표면을 부식으로부터 보호하는 데 도움이 됩니다. 이러한 방청 페인트의 일반적인 유형은 다음과 같습니다:1. 에폭시 방청 페인트: 내구성이 뛰어나고 부식에 강한 에폭시 방청 페인트는 자동차 부품, 도금된 금속 표면, 배 등 다양한 표면에 사용됩니다.2. 아크릴 방청 페인트: 아크릴 방청 페인트는 저렴하고 쉽게 적용할 수 있어 다양한 상업용 및 주거용 응용 프로그램에 사용됩니다.3. 알키드 방청 페인트: 알키드 방청 페인트는 내구성이'

In [ ]:
generate_response(test['질문'][0],trained_solar_model, solar_tokenizer, 400)

'방청 페인트는 방청 특성을 가진 페인트의 일종입니다. 방청 페인트는 대기 중의 산화 및 부식을 방지하기 위해 특별히 설계되었습니다. 방청 페인트의 일반적인 유형에는 다음이 있습니다:1. 에폭시 방청 페인트: 강도, 내마모성, 내산화성이 뛰어난 고도의 방청 특성을 제공하는 에폭시 방청 페인트가 있습니다. 이러한 페인트는 자동차, 건물, 선박과 같은 산업 및 상업 애플리케이션에서 자주 사용됩니다.2. 아크릴 방청 페인트: 아크릴 방청 페인트는 에폭시 방청 페인트와 유사한 방청 특성을 제공하지만 가격이 저렴하고 쉽게 사용할 수 있습니'

In [ ]:
!pip install gensim

In [ ]:
from gensim.summarization import summarize

def extractive_summarization(text, ratio=0.2):
    summary = summarize(text, ratio=ratio)
    return summary

# 사용 예시
original_text = "방청 페인트는 방청 성분이 포함된 특별한 종류의 페인트입니다. 방청 성분은 철과 같은 금속 표면을 부식으로부터 보호하는 데 도움이 됩니다. 이러한 방청 페인트의 일반적인 유형은 다음과 같습니다:1. 에폭시 방청 페인트: 내구성이 뛰어나고 부식에 강한 에폭시 방청 페인트는 자동차 부품, 도금된 금속 표면, 배 등 다양한 표면에 사용됩니다.2. 아크릴 방청 페인트: 아크릴 방청 페인트는 저렴하고 쉽게 적용할 수 있어 다양한 상업용 및 주거용 응용 프로그램에 사용됩니다.3. 알키드 방청 페인트: 알키드 방청 페인트는 내구성이"
summary_text = extractive_summarization(original_text)
print("Original Text:\n", original_text)
print("\nExtractive Summary:\n", summary_text)


ModuleNotFoundError: No module named 'gensim.summarization'

In [ ]:
generate_response(test['질문'][3],base_model, solar_tokenizer, 400)

'철골 구조를 사용하는 고층 건물의 단열 효과를 높이기 위해 몇 가지 시공 방법을 사용할 수 있습니다. 이러한 방법은 다음과 같습니다:1. 단열재 사용: 건물 내부 및 외부에 단열재를 사용하여 단열 효과를 높일 수 있습니다. 단열재는 건물 내부와 외부에 공기 또는 기타 단열 물질을 사용하여 열 전달을 방지합니다.2. 외벽 단열재: 외벽 단열재는 건물의 외부 벽에 추가 단열을 제공하여 열 전달을 방지하는 데 도움이 됩니다. 이러한 단열재는 다양한 재료와 방식으로 제공되며, 건물의 특정 요구 사항에 맞게 선택할 수 있습니다.3. 기밀성'

## 2) Solar Fewshot Model

## 3) Solar Pre-trained Model

In [ ]:

# 학습 후 peft 모델
adapter_model_name = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/models/megast_0302/'
model = PeftModel.from_pretrained(base_model, adapter_model_name).to(device)
print(model)
print('#'*100)

# 학습 후 merge된 peft 모델
merge_model = model.merge_and_unload()
print(merge_model)


# Rule Based Similarity (Solar - KoGPT 분리 기준)

## 1) BLEU score

## 2) Cosine / TFIDF